In [30]:
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler
from sklearn.linear_model import LinearRegression, Ridge
from scipy.optimize import minimize

def get_trained_models(X, Y):
    pipe = Pipeline([
        ('scaler', MinMaxScaler(feature_range=(0, 1))),
        ('poly', PolynomialFeatures(degree=5)),
        ('lr', Ridge(alpha=1.0))
    ])

    target_transform = MinMaxScaler(feature_range=(0, 1))
    
    target_transform.fit(Y)
    pipe.fit(X, target_transform.transform(Y))
    return pipe, target_transform

def minimize_maximize(pipe, target_transform, boundary):
    startpoint = np.array([np.mean(b) for b in boundary])
    min_functions = []
    min_values = []
    for i in range(len(target_transform.data_range_)):
        min_functions.append(lambda x: pipe.predict([x])[:, i])
        min_values.append(target_transform.inverse_transform(pipe.predict([minimize(min_functions[i], x0=startpoint, bounds=boundary).x]))[0, i])
        
    max_functions = []
    max_values = []
    for i in range(len(target_transform.data_range_)):
        max_functions.append(lambda x: -pipe.predict([x])[:, i])    
        max_values.append(target_transform.inverse_transform(pipe.predict([minimize(max_functions[i], x0=startpoint, bounds=boundary).x]))[0, i])
    
    return [(min_v, max_v) for min_v, max_v in zip(min_values, max_values)]

def find_Xi_pareto(pipe, target_transform, X_boundary, Y_desired_boundary, target, steps=100):
    ls = [np.array([np.linspace(b[0], b[1], num=steps) if i != target else np.ones(steps)*v for i, b in enumerate(X_boundary)]).T for v in np.linspace(X_boundary[target][0], X_boundary[target][1], steps)]
    results = []
    for l in ls:
        preds = target_transform.inverse_transform(pipe.predict(l))
        results.append({'min': preds.min(axis=0), 'max': preds.max(axis=0)})

    filtered_results = []

    for r, value in zip(results, np.linspace(X_boundary[target][0], X_boundary[target][1], steps)):
        if all([r['min'][ind] >= y[0] and r['max'][ind] <= y[1] for ind, y in enumerate(Y_desired_boundary)]):
            filtered_results.append(value)
            
    return filtered_results

def find_X_pareto(pipe, target_transform, X_boundary, Y_desired_boundary, X_targets, steps=100, multipl=0.75, iteration=0):
    print(iteration)
    if iteration >= 10:
        return 'Empty'
    results = []
    for target in X_targets:
        results.append(find_Xi_pareto(pipe, target_transform, X_boundary, Y_desired_boundary, target, steps=100))
    if any([len(r) for r in results]):
        ind_corrected = np.argmax([len(r) for r in results])
        target_corrected = X_targets[ind_corrected]
        X_boundary_new = [b if ind != target_corrected else (results[ind_corrected][0], results[ind_corrected][-1]) for ind, b in enumerate(X_boundary)]
        return X_boundary_new
    else:
        X_boundary_new = [b if ind not in X_targets else (np.mean(b) - multipl*(np.mean(b) - b[0]), np.mean(b) + multipl*(b[1] - np.mean(b))) for ind, b in enumerate(X_boundary)]
        return find_X_pareto(pipe, target_transform, X_boundary_new, Y_desired_boundary, X_targets, steps=steps*multipl, multipl=multipl, iteration=iteration + 1)
    
def get_results(inp):
    X = inp['X_values']
    Y = inp['Y_values']
    X_boundary = [(x['min'], x['max']) for x in inp['X']]
    Y_desired_boundary = [(y['min'], y['max']) for y in inp['Y']]
    poly = np.quantile(inp['poly'], 0.75)
    action = inp['action']
    X_targets = np.where([x['correct'] for x in inp['X']])[0]
    
    pipe, target_transform = get_trained_models(X, Y)
    real_bounds = minimize_maximize(pipe, target_transform, X_boundary)
    if action == 'Y':
        return real_bounds
    else:
        return find_X_pareto(pipe, target_transform, X_boundary, Y_desired_boundary, X_targets, steps=100)

In [43]:
import pandas as pd
import numpy as np

X = pd.read_csv('SunX.txt', header=None)
Y = pd.read_csv('SunY.txt', header=None)

X_boundary = [
    (xmin, xmax) for xmin, xmax in zip(X.min(axis=0).tolist(), X.max(axis=0).tolist())
]

Y_boundary = [
    (xmin, xmax) for xmin, xmax in zip(Y.min(axis=0).tolist(), Y.max(axis=0).tolist())
]

X_targets = [0, 1, 2, 3]
Y_desired_boundary = [(-50, 15), (-50, 15)]

X_boundary, Y_boundary

([(0.287, 0.384),
  (5.6, 16.0),
  (-14.6, 4.8),
  (0.0, 4.745),
  (-60.0, 17.0),
  (-60.0, 17.0)],
 [(-60, 17), (-60, 17)])

In [44]:
inp = {'X_values': X, 'Y_values': Y, 'X': [{'min': x[0], 'max': x[1], 'correct': True if ind in X_targets else False} for ind, x in enumerate(X_boundary)], \
       'Y': [{'min': x[0], 'max': x[1], 'correct': False} for ind, x in enumerate(Y_desired_boundary)], 'poly': [1, 3, 4], 'action': 'X'}

In [45]:
get_results(inp)

0
1


[(0.299125, 0.371875),
 (6.9, 14.7),
 (-12.175, -9.382575757575758),
 (0.5931250000000001, 4.151875),
 (-60.0, 17.0),
 (-60.0, 17.0)]

In [32]:
Y_boundary

[(-60, 17), (-60, 17)]

In [29]:
0.75**10

0.056313514709472656

In [21]:
X_boundary

[(0.287, 0.384),
 (5.6, 16.0),
 (-14.6, 4.8),
 (0.0, 4.745),
 (-60.0, 17.0),
 (-60.0, 17.0)]

In [ ]:
{'X_values': pd.DataFrame, 'Y_values': pd.DataFrame, 'X' [{'min': float, 'max': float, 'correct': bool}], 'Y' [{'min': float, 'max': float, 'correct': bool}] 'poly': [int], 'action': str}

In [11]:
np.where([True, False, True, False])[0]

array([0, 2])

In [3]:
pipe, target_transform = get_trained_models(X, Y)

In [4]:
real_bounds = minimize_maximize(pipe, target_transform, X_boundary)

In [5]:
real_bounds

[(-83.4268729772238, 68.28235015725654),
 (-75.7953935139678, 70.88512416316503)]

In [6]:
Y_desired_boundary = [(-83, 50), (-70, 60)]

In [7]:
for target in X_targets:
    print(find_Xi_pareto(pipe, target_transform, X_boundary, Y_desired_boundary, target, steps=100))

[0.287, 0.28797979797979795, 0.2889595959595959, 0.28993939393939394, 0.2909191919191919, 0.2918989898989899, 0.29287878787878785, 0.2938585858585858, 0.29483838383838384, 0.2958181818181818, 0.2967979797979798, 0.29777777777777775, 0.2987575757575757, 0.29973737373737375, 0.3007171717171717, 0.3016969696969697, 0.30267676767676766, 0.3036565656565656, 0.3046363636363636, 0.3056161616161616, 0.3065959595959596, 0.30757575757575756, 0.3085555555555555, 0.3095353535353535, 0.3105151515151515, 0.3114949494949495, 0.31247474747474746, 0.31345454545454543, 0.3144343434343434, 0.3154141414141414, 0.3163939393939394, 0.31737373737373736, 0.31835353535353533, 0.3193333333333333, 0.3203131313131313, 0.3212929292929293, 0.32227272727272727, 0.32325252525252524, 0.3242323232323232, 0.3252121212121212, 0.3261919191919192, 0.32717171717171717, 0.32815151515151514, 0.3291313131313131, 0.3301111111111111, 0.3310909090909091, 0.33207070707070707, 0.33305050505050504, 0.334030303030303, 0.3350101010101

In [33]:
steps = 100

In [34]:
steps = 100

for target in X_targets:
    ls = [np.array([np.linspace(b[0], b[1], num=steps) if i != target else np.ones(steps)*v for i, b in enumerate(X_boundary)]).T for v in np.linspace(X_boundary[target][0], X_boundary[target][1], steps)]
    
    results = []
    for l in ls:
        preds = target_transform.inverse_transform(pipe.predict(l))
        results.append({'min': preds.min(axis=0), 'max': preds.max(axis=0)})

    filtered_results = []

    for r, value in zip(results, np.linspace(X_boundary[target][0], X_boundary[target][1], steps)):
        if all([r['min'][ind] >= y[0] and r['max'][ind] <= y[1] for ind, y in enumerate(Y_desired_boundary)]):
            filtered_results.append(value)
    print(filtered_results)
    print()

[0.287, 0.28797979797979795, 0.2889595959595959, 0.28993939393939394, 0.2909191919191919, 0.2918989898989899, 0.29287878787878785, 0.2938585858585858, 0.29483838383838384, 0.2958181818181818, 0.2967979797979798, 0.29777777777777775, 0.2987575757575757, 0.29973737373737375, 0.3007171717171717, 0.3016969696969697, 0.30267676767676766, 0.3036565656565656, 0.3046363636363636, 0.3056161616161616, 0.3065959595959596, 0.30757575757575756, 0.3085555555555555, 0.3095353535353535, 0.3105151515151515, 0.3114949494949495, 0.31247474747474746, 0.31345454545454543, 0.3144343434343434, 0.3154141414141414, 0.3163939393939394, 0.31737373737373736, 0.31835353535353533, 0.3193333333333333, 0.3203131313131313, 0.3212929292929293, 0.32227272727272727, 0.32325252525252524, 0.3242323232323232, 0.3252121212121212, 0.3261919191919192, 0.32717171717171717, 0.32815151515151514, 0.3291313131313131, 0.3301111111111111, 0.3310909090909091, 0.33207070707070707, 0.33305050505050504, 0.334030303030303, 0.3350101010101